# Shaft water rise monitoring




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn")
import numpy as np
import matplotlib.dates as mdates
import pickle

from my_func_mvw.functions_import_my_database import import_my_database_pickle, import_my_database_csv, merge_data_year, import_tlogger
from my_func_mvw.functions import find_nearest_date, calc_diff_between_channels, read_pickle, write_pickle, temp_watertank_func, random_date,  save_values_in_file, plot_water_rise, create_mask_egrt 
from my_func_mvw.functions_dts_processing import watertank_shift, check_first_last_date, cut_dataframe_to_range_tlogger, check_processed_data, diff_to_watertank
%load_ext autoreload
%autoreload 2

### Input
plot_save=True #True False
################################
path_DTS_processed = r"..\Alsdorf\Daten\DTS_processed"
masterthesis_save=False # True False
importer="pickle" #pickle currently only pickle is uesed 

print("Some Version information of the imported packages")
print(f"pandas version: {pd.__version__}")
print(f"pickle version: {pickle.format_version}")

Some Version information of the imported packages
pandas version: 1.3.4
pickle version: 4.0


In [2]:
# Import data

# Channels 1 to 4
shaft={}
for chan in ["1","2","3","4"]:
    shaft[chan] = read_pickle(path_DTS_processed + r"\shaft_temperatures\old_cable\pickle" + f"\\Schacht_chan_{chan}")

# EGRT cable
Schacht_7and8_0_down = read_pickle(path_DTS_processed + r"\shaft_temperatures\egrt_cable\pickle" + "\\Schacht_7and8_down")
Schacht_7and8_0_up = read_pickle(path_DTS_processed + r"\shaft_temperatures\egrt_cable\pickle" + "\\Schacht_7and8_up")

## Channels 1 to 4

In [3]:
# Input #####
channel="2"
resample_hours=12
start_date=1 #1720; #1720can be used to get a better comparisson with channels 5 to 8
#use with show_additional_water_level_info=False in plot function
##################

col=shaft[channel].columns[145:170]
resample_data=shaft[channel][col].resample(f"{resample_hours}H").mean()[start_date:]

col=resample_data.columns[4:] # so both plots have same shape in thesis

#plot and save
plot_water_rise(resample_data[col],linear_curve=[163.3,0.008],plot_save=masterthesis_save,show_additional_water_level_info=True)

fitted water level rise: 2.9 m / year
this is a total of 8.0 m in the measurement time


In [4]:
# Calc diff and determine linear curve at that plot
# makes it a bit easier to observe the transition

# Calc diff
all_diff_columns=[]
number_of_averages=1 #averages depths, unenecessary just use 1
for x in resample_data.columns[:-number_of_averages*2]:
    x_index=x-resample_data.columns[0]
    col1=resample_data.columns[x_index : x_index+number_of_averages]
    col2=resample_data.columns[x_index+number_of_averages : x_index+2*number_of_averages]
    diff_colum=resample_data[col2].mean(axis=1) - resample_data[col1].mean(axis=1)
    #diff_colum=resample_data[x] - resample_data[x+1]

    all_diff_columns.append(diff_colum)

df_diff=pd.concat(all_diff_columns,axis=1)
df_diff.columns=resample_data.columns[:-number_of_averages*2]+number_of_averages+0.5
df_diff=round(df_diff,1)

# Plot and save
plot_water_rise(df_diff,plot_save=masterthesis_save,linear_curve=[163.3,0.008],title="Diff.",zminmax=[0,0.5],show_additional_water_level_info=True)

fitted water level rise: 2.9 m / year
this is a total of 8.0 m in the measurement time


## Channels 5 to 8

In [5]:
# Resample data
resample_hours=12
col=Schacht_7and8_0_down.columns[145:175]
resample_data=Schacht_7and8_0_down[col].resample(f"{resample_hours}H").mean()

# so both plots have same shape in thesis
col=resample_data.columns[4:]
resample_data[col]

plot_water_rise(resample_data[col],plot_save=masterthesis_save,linear_curve=[156.5,0.008],data_type="chan58",show_additional_water_level_info=False)

fitted water level rise: 2.9 m / year
this is a total of 1.2 m in the measurement time


In [6]:
# Calc diff and determine linear curve at that plot
all_diff_columns=[]
number_of_averages=1
for x in resample_data.columns[:-number_of_averages*2]:
    noa=number_of_averages
    x_index=x-resample_data.columns[0]
    col1=resample_data.columns[x_index:x_index+noa]
    col2=resample_data.columns[x_index+noa:x_index+2*noa]
    diff_colum=resample_data[col2].mean(axis=1) - resample_data[col1].mean(axis=1)
    #diff_colum=resample_data[x] - resample_data[x+1]

    all_diff_columns.append(diff_colum)

df_diff=pd.concat(all_diff_columns,axis=1)
df_diff.columns=resample_data.columns[:-number_of_averages*2]+number_of_averages+0.5
df_diff=round(df_diff,1)

# Plot and save
plot_water_rise(df_diff,plot_save=masterthesis_save,linear_curve=[156.5,0.008],title="Diff.",zminmax=[0,0.5],data_type="chan58",show_additional_water_level_info=False)

fitted water level rise: 2.9 m / year
this is a total of 1.2 m in the measurement time


In [7]:
# Try to find a function which describes shaft water rise with data from rosner2011
# bad results
show_interpolation=False 
if show_interpolation:
    alph=0.4 #for some curves
    # very large time range for plotting
    start=pd.to_datetime("01.01.1990")
    end=pd.to_datetime("01.01.2050")
    x_large=mdates.date2num(pd.date_range(start,end,freq="M"))
    fig,axs=plt.subplots(1,1,figsize=(16,5))

    #Quadratic fit
    first_date=pd.to_datetime("1994-01-01")
    last_date=pd.to_datetime("2022-01-01")
    dates=pd.date_range(first_date,last_date,freq="D")
    datetonum=mdates.date2num(dates)
    a=0.000006 #0.000005
    h=datetonum[-1] #last_date
    k=155 # y position of final water level
    print("Annahme Scheitelpunkt is am last_date erreicht erreicht, für manual fit")
    x=datetonum-datetonum[0]
    ground_water_depth_fit_quadratic=a*(x-h)**2+k
    x=x_large
    axs.plot(x,a*(x-h)**2+k,label="quadratic fit manual",linestyle="--",alpha=alph) 

    #linear fit
    datetonum_linear=mdates.date2num(df_diff.index)
    x=datetonum_linear-datetonum_linear[0]
    ground_water_depth_fit_linear=-0.008*x+163
    axs.plot(datetonum_linear,ground_water_depth_fit_linear,label="linear fit from DTS",linewidth=9)

    # scatter measured data
    dates_scat=["01.01.1995","01.05.1998","01.01.2002","04.12.2003","01.01.2005","01.01.2009","24.11.2020"]
    #"04.12.2003" wireline log 2003
    #"24.11.2020" wireline log 2020
    #Rest von rosner 2011 abgeschätzt aus Abbildung
    depth_scat=[860,610,460,392,360,252,158]
    axs.scatter(pd.to_datetime(dates_scat),depth_scat,label="measured water level?",zorder=10)

    # fitted quadratic function from scatter data
    datetonum_q_fit=mdates.date2num(pd.to_datetime(dates_scat))
    x=datetonum_q_fit
    poly=np.polyfit(x[1:],depth_scat[1:],2)
    x=x_large
    axs.plot(x,poly[2]+ x*poly[1]+x**2*poly[0],label="fitted quadratic from scatter points\nneglet oldest point",linestyle="--",alpha=alph)
    x=datetonum_q_fit
    poly=np.polyfit(x,depth_scat,2)
    x=x_large
    axs.plot(x,poly[2]+ x*poly[1]+x**2*poly[0],label="fitted quadratic from scatter points",linestyle="--",alpha=alph)

    #fitted quadratic from linear and wireline logs
    x_dates=pd.to_datetime([pd.to_datetime("04.12.2003")]).union(np.array(df_diff.index[::400]))
    datetonum_linearploly=mdates.date2num(x_dates)
    poly=np.polyfit(datetonum_linearploly,np.append(392,ground_water_depth_fit_linear[::400]),2)
    x=x_large
    axs.plot(x,poly[2]+ x*poly[1]+x**2*poly[0],label="fitted quadratic from linear and wireline 2003 scatter point",linestyle="--",alpha=alph)

    datetonum_linearploly=mdates.date2num(df_diff.index[::400])
    poly=np.polyfit(datetonum_linearploly,ground_water_depth_fit_linear[::400],2)
    x=x_large
    axs.plot(x,poly[2]+ x*poly[1]+x**2*poly[0],label="fitted quadratic, only from linear",linestyle="--",alpha=alph)

    # fit an asymtotic function
    def asy_function(x, a,b):
        """asympotic function"""
        return a+b/x
    from scipy.optimize import curve_fit
    # # fit only with scattered data
    # popt, pcov = curve_fit(asy_function, datetonum_q_fit, depth_scat)
    # add some data of linear curve to fit
    datetonum_asyploly=mdates.date2num(df_diff.index[::400]).tolist()
    all_datetonum_i_want=datetonum_asyploly+datetonum_q_fit.tolist()[2:] #scip oldest points
    all_depths_I_want=ground_water_depth_fit_linear[::400].tolist()+depth_scat[2:]
    popt, pcov = curve_fit(asy_function, all_datetonum_i_want, all_depths_I_want)
    x=x_large
    axs.plot(x,asy_function(x,popt[0],popt[1]),label=f"asymptotic fit to scattered and linear data\nasymtope: {round(popt[0],1)} m",linestyle="--",alpha=alph)

    # # semi manuall fitted asymptote, does not give good resutls
    # # also tried looping over different asymptote constants
    # def asy_function_semi_fit(x ,b):
    #     """asympotic function"""
    #     asy_const=130
    #     return asy_const+b/x
    # popt, pcov = curve_fit(asy_function_semi_fit, all_datetonum_i_want, all_depths_I_want)
    # axs.plot(x,asy_function_semi_fit(x,popt[0]),label=f"asymptotic semi fit to scattered and linear data")#),linestyle="--",alpha=alph)

    #e-function fit #cant find optimal parameters
    # def e_function(x, a,b): 
    #     """asympotic function"""
    #     return a*np.e**(x+b)

    # Assecoirs
    axs.set_ylim(100,800)
    axs.invert_yaxis()
    axs.legend()
    plt.show()